In [2]:
import os, sys
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator
import heapq as hq
import torch

from tqdm.notebook import trange
from tqdm import tqdm

# Init steps

In [3]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks : helpers.ipynb contains a set of tools and functions used in this notebook
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

In [4]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

# Prepare models/datasets

In [5]:
# Load the pretrained models "lstm" (entire_model_1m_20interactions.pt) and "pooling" (pooling_model_1m_20interactions.pt) presents in the models folder

# implicit_model = load_model('implicit_factorization')
lstm_model = load_model(model_type='entire') # the code to create this model is in misc.ipynb (see section "train model")
pooling_model = load_model('pooling') # the code to create this model is in brute_force_rec_expl.ipynb (see section "train and save pooling model")

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

In [6]:
# Get the dataset Movielens with the variant 1M. Then divide it into a training set and a testing set. It also limits the length of each sequence of elements in the 2 sets to 20.

from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset, more information here : https://grouplens.org/datasets/movielens/
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20 # Maximum sequence length. Subsequences shorter than this will be left-padded with zeros.
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length) # test is a SequenceInteractions object, here is the documentation on this object : https://maciejkula.github.io/spotlight/interactions.html

In [7]:
# Compute cosine similarity matrix for item embeddings using GPU
pooling_sims_matrix = gpu_embeddings_to_cosine_similarity_matrix(
    pooling_model._net.item_embeddings(
        torch.arange(0, dataset.num_items, dtype=torch.int64)
    )).detach().numpy()

# Compute item-item similarity matrix using Jaccard similarity
jaccard_sims_matrix = compute_sim_matrix(dataset, 'jaccard')

  0%|          | 0/6040 [00:00<?, ?it/s]

In [7]:
print(jaccard_sims_matrix[0])
print(jaccard_sims_matrix[1])
print(jaccard_sims_matrix[0].shape)
print(jaccard_sims_matrix[1].shape)

[1.00000000e+00 1.03482099e-01 1.48905109e-01 ... 0.00000000e+00
 5.79710145e-04 0.00000000e+00]
[0.1034821  1.         0.13600783 ... 0.         0.         0.        ]
(3706,)
(3706,)


In [8]:
print(max(dataset.user_ids))
print(min(dataset.user_ids))
print(dataset.num_users)
print(max(dataset.item_ids))
print(min(dataset.item_ids))
print(dataset.num_items)

6040
1
6041
3706
1
3707


# Various implemented Strategies

In [8]:
class BaseStrategy:
    class_name = None

    def __init__(self, item, interactions, max_length, init_budget,  model=None, random_pick=False):

        self.target_item = item
        self.original_interactions = interactions
        self.max_length = max_length
        self.visited_ = set()
        self.model = model
        self.last_comb_cost = 0
        self.random_pick = random_pick
        self.top_k = 10
        self.budget = init_budget

    # Must be implemented by subclasses. Used to select the next item to recommand to the user.
    def next_comb(self, reverse=False):
        raise NotImplementedError

    
    # This method takes a "number" argument and returns a list of 0-bit positions in the binary representation of "number".
    def _get_pos(self, number):
        bits = []
        for i, c in enumerate(bin(number)[:1:-1], 1):
            if c == '0':
                bits.append(i)
        return bits

    # Method to reset the costs of the last recommended combination
    def reset_costs(self):
        self.last_comb_cost = 0

    # Returns the initial budget
    def get_init_budget(self):
        return self.budget

### RandomSelection
#### This class is a subclass of the "BaseStrategy" class, representing a random item selection strategy for the sequential recommendation task.

In [9]:
class RandomSelection(BaseStrategy):
    class_name = 'Random'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget)

    # The _next_item method selects a random integer between 1 and 2 raised to the power of the maximum length of the sequence. 
    # It checks if the integer is already in the set of visited integers, and if it is, selects another integer until a non-visited integer is found. 
    # Finally, the selected integer is added to the set of visited integers, and returned.
    def _next_item(self):
        self.budget -= 1
        
        number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        while number in self.visited_:
            number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        self.visited_.add(number)
        
        return number
    
    # The next_comb method generates a new sequence by removing items at positions indicated by the binary digits of the integer returned 
    # by _next_item from the original sequence of interactions. 
    # The resulting sequence, along with the current budget, is returned as a tuple.
    def next_comb(self, reverse=False):
        number = self._next_item()

        bits = self._get_pos(number)
        seq = np.delete(self.original_interactions, bits)

        return (seq, self.budget)

### LossSimilarSelection
#### This class inherits from the BaseStrategy class. This class defines a search strategy for selecting items based on their similarity to previously selected items, while also considering their loss (difference between predicted and actual values) in a ranking problem.

In [10]:
class LossSimilarSelection(BaseStrategy):
    class_name = 'BFS'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, early_term=False):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        mask = [False] * len(self.original_interactions)
        t_score = StaticVars.INT_MAX
        is_solved = 0

        self.q = Queue()
        self.q.enqueue((mask, t_score, is_solved))

        self.thres = len(self.original_interactions) + 1
        self.early_termination = early_term

    # Helper : This method is called whenever a solution is found for the current 
    # mask of the items. It computes the loss of the solution and updates the queue 
    # accordingly.
    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    # Helper : 
    def _next_item(self):
        mask, t_score, is_solved = self.q.dequeue()
        while self.early_termination and sum(mask) == self.thres:
            q_data = self.q.dequeue()
            if q_data is None: 
                break
            mask, t_score, is_solved = q_data

        if is_solved == 2:
            t_score, kth_score = self.get_score(mask)

            if (t_score / kth_score) < 1: 
                self.thres = sum(mask)

        return (is_solved, mask, self.budget)

    # The next_comb method returns the next combination of items and the remaining budget.
    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q.size() > 0:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy()) # If there is a True in the mask, the value at the corresponding index is masked
            self._update_queue(solved_flag)
        else: 
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr) #Extract all the valid values in ma_arr
        return (seq, budget) if len(seq) else (None, budget)

    # This method computes the loss of the solution. If the solution is not yet solved, it searches for the next combination of items 
    # to evaluate by calling the search method. 
    # If the solution is solved, it searches for the previous combination of items by calling the search method with forward=False.
    def compute_loss(self, is_solved=False):
        self.last_comb_cost = 0

        if not is_solved: 
            self.search(forward=True, s=is_solved)#
        else: 
            self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        """
        Searches for the next combination of items based on the search direction.

        Args:
            forward (bool, optional): If True, the search is performed forward, otherwise backward.
            s (bool, optional): Indicates whether a solution is found.
        """
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0] #if forward is True it means we keep index of items where the m_mask is not True
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_score(self, d):
        """
        Computes the score of an item based on its predicted value and rank among the top-k items.

        Args:
            d (array_like): Boolean mask indicating the selected items.

        Returns:
            Tuple: A tuple containing the score of the target item and its rank among the top-k items.
        """
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

        self.budget -= 1
        # predict next top-k items about to be selected
        preds = self.model.predict(perm)
        preds[perm] = -StaticVars.FLOAT_MAX
        rk_data = st.rankdata(-preds, method='ordinal')

        return (preds[self.target_item], preds[(rk_data == self.top_k).nonzero()][0])

    def add(self, d, s):
        """
        Adds a new combination to the queue and updates the visited set.

        Args:
            d (array_like): Boolean mask representing the combination of items.
            s (bool): Indicates whether a solution is found.
        """
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

            if not s:
                t_score, kth_score = self.get_score(d)

                if self.q.size() == 0: self.q.enqueue((d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))

                if t_score < self.q.get(0)[1]:  # get only the assigned score
                    self.q.setter(0, (d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))
            else:
                self.q.enqueue((d.copy(), StaticVars.INT_MAX, 2))

            self.visited_.add(mask_to_int)

### BiDirectionalSelection

In [11]:
class BiDirectionalSelection(BaseStrategy):
    class_name = 'BiDirectional'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.tiebraker = itertools.count()
        self.q = [(1, StaticVars.INT_MAX, next(self.tiebraker), [False] * len(self.original_interactions), self.budget)]
        hq.heapify(self.q)

        self.alpha = alpha
        self.norm = normalization

    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    def _next_item(self):
        is_solved, _, _, mask, budget = hq.heappop(self.q)
        return (is_solved, mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget
        if self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def compute_loss(self, is_solved=False):
        self.search(forward=True, s=is_solved)
        self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_custom_score(self, c):
        return c / self.max_length

    def get_score(self, d):
        self.budget -= 1

        # predict next top-k items about to be selected
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))
        preds = self.model.predict(perm)

        if self.norm == 'kth_norm':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            t_score = preds[self.target_item] / preds[(rk_data == self.top_k).nonzero()][0]
        elif self.norm == 'rescale':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            max_val = rk_data[0]
            min_val = rk_data[-1]
            t_score = (max_val - preds[self.target_item]) / (max_val - min_val)
        else:  # default case
            tensor = F.softmax(torch.from_numpy(preds).float(), dim=0)
            preds = tensor.numpy()
            preds[perm] = -StaticVars.FLOAT_MAX

            t_score = preds[self.target_item]

        return self.alpha * t_score + (1 - self.alpha) * self.get_custom_score(np.sum(d))

    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            t_score = self.get_score(d)
            hq.heappush(self.q, (int(not s), t_score, next(self.tiebraker), d.copy(), self.budget))

            self.visited_.add(mask_to_int)

# BruteForceSelection

In [12]:
class BruteForceSelection(BaseStrategy):
    class_name = 'BruteForce'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), self.budget))

    def _expand_queue(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask))[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask)
                m_mask[idx] = not m_mask[idx]

    def _next_item(self):
        mask, budget = self.q.dequeue()
        return (mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget

        if reverse: self.q.clear()

        if self.q.size() > 0:
            item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._expand_queue()
        else:
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def add(self, d):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            self.budget -= 1
            self.q.enqueue((d.copy(), self.budget))
            self.visited_.add(mask_to_int)

# ComboSelection

In [13]:
class ComboSelection(BiDirectionalSelection):
    class_name = 'Combo'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model, weights, alpha, normalization)

        self.alpha = 1

        self.q_init = Queue()
        self.q_init.enqueue((StaticVars.INT_MAX, [False] * len(self.original_interactions), self.budget))
        self.init_queue()

        self.tiebraker = itertools.count()
        self.q = []
        hq.heapify(self.q)

        self.alpha = alpha

    def init_queue(self):
        _, m_mask, budget = self.q_init.dequeue()
        m_mask = np.asarray(m_mask)

        valid_items = np.where(np.logical_not(m_mask))[0]
        for idx in valid_items:
            m_mask[idx] = not m_mask[idx]

            mask_to_int = int(''.join(map(str, m_mask.astype(int))), 2)
            if (mask_to_int not in self.visited_) and (self.budget > 0):
                t_score = self.get_score(m_mask)
                self.q_init.enqueue((t_score, m_mask.copy(), self.budget))

                self.visited_.add(mask_to_int)

            m_mask[idx] = not m_mask[idx]

        pair_combs = []
        for c in itertools.combinations(range(len(self.original_interactions)), 2):
            m = [False] * len(self.original_interactions)
            m[c[0]], m[c[1]] = not m[c[0]], not m[c[1]]
            pair_combs.append((self.q_init.get(c[0])[0] + self.q_init.get(c[1])[0], m.copy()))

        pair_combs.sort(key=operator.itemgetter(0))
        for c in pair_combs:
            self.budget -= 1
            self.q_init.enqueue((0, c[1], self.budget))

    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q_init.size() > 0:
            s, item_mask, budget = self.q_init.dequeue()
            item_mask = np.asarray(item_mask)
            solved_flag = False
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self.add(item_mask, False)
        elif self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

### Temporary code

In [14]:
class MostSimilarSelection(BaseStrategy):
    class_name = 'Sim-Matrix'

    supported_sim_matrix = {
        'pooling': pooling_sims_matrix,
        'jaccard': jaccard_sims_matrix
    }

    def __init__(self, item, interactions, max_sequence_length, model, sim_type='pooling'):
        super().__init__(item, interactions, max_sequence_length)

        self.visited_.add(0)
        self.reverse_checks = []
        self.is_materialized = False

        self._get_sim_ranking(sim_type)

    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
            selected_item_indices = self.reverse_checks.pop(
                random.randrange(len(self.reverse_checks)) if self.random_pick else 0
            ) if len(self.reverse_checks) else []
        else:
            self.visited_.add(max(self.visited_) + 1)
            selected_item_indices = np.where(np.isin(
                self.rk_items,
                list(set(self.rk_items).difference(set(self.visited_)))
            ))[0]
        seq = self.original_interactions[selected_item_indices] if len(selected_item_indices) else None
        return seq

    def _get_sim_ranking(self, sim_type):
        ranked_items = st.rankdata(self.supported_sim_matrix[sim_type][self.target_item, self.original_interactions])
        self.rk_items = self.max_length - ranked_items + 1

    def _materialize_list(self):
        if not self.is_materialized:
            psize = len(self.visited_) - 1  # do not consider initial added zero value
            # do not take account none/all excluded interacted items
            prods = sorted(list(map(list, itertools.product([0, 1], repeat=psize)))[1:-1], key=sum)
#             last_item_indices = np.where(np.isin(
#                 self.rk_items,
#                 list(set(self.rk_items).difference(set(self.visited_)))
#             ))

            lvisited_ = np.asarray(list(self.visited_))[1:]
            for p in prods:
                self.reverse_checks.append(np.where(np.isin(
                    self.rk_items,
                    list(set(self.rk_items).difference(lvisited_[np.nonzero(np.multiply(p, lvisited_))])))
                ))

            self.is_materialized = True

In [18]:
class MostSimilarSelectionByJaccard(MostSimilarSelection):
    class_name = 'Jaccard-on-Sim-Matrix'

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model, 'jaccard')

In [17]:
# cosine_on_embeddings_cfs = [
#     _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000)
# ]
# jaccard_on_embeddings_cfs = [
#     _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
# ]

# %store cosine_on_embeddings_cfs
# %store jaccard_on_embeddings_cfs

# Get backend strategy

In [14]:
def get_backend_strategy(backend):
    if 'random' == backend:
        return RandomSelection
    elif 'most_sim' == backend:
        return MostSimilarSelection
    elif 'most_sim_jaccard' == backend:
        return MostSimilarSelectionByJaccard
    elif 'bfs' == backend:
        return LossSimilarSelection
    elif 'random_most_sim' == backend:
        return RandomMostSimilarSelection
    elif 'random_loss_sim' == backend:
        return RandomLossSimilarSelection
    elif 'fixed_loss_sim' == backend:
        return FixedRankingLossSimilarSelection
    elif 'dfs_loss_sim' == backend:
        return DFSwithLossSelection
    elif 'dfs_fixed_loss_sim' == backend:
        return DFSwithFixedRankingLossSelection
    elif 'bestFS_loss' == backend:
        return BestFSLossSelection
    elif 'bestFS_fixed_loss' == backend:
        return BestFSFixedLossSelection
    elif 'topdown_loss' == backend:
        return TopDownBestFSLossSelection
    elif 'bidirectional' == backend:
        return BiDirectionalSelection
    elif 'brute_force' == backend:
        return BruteForceSelection
    elif 'combo' == backend:
        return ComboSelection
    else: print('Unknown strategy')

# Run implemented strategies

In [22]:
import numpy as np
from collections import defaultdict
import tqdm
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

def replace_items_if_missing(items_removed, target_list):
    items_to_replace = set(items_removed) - set(target_list)
    for i, item in enumerate(items_to_replace):
        j = i
        while target_list[j] in items_removed:
            j += 1
        target_list[j] = item

def create_reverse_mode_evaluation_dataframe(dataset, model, strategy, target_item_pos, top_k, sim_matrix):
    evaluation_df = pd.DataFrame(columns=['user_id', 'target_pos', 'cfs', 'len_cfs',])
    len_sample = 20

    for user_id in tqdm.notebook.tqdm(range(1, max(dataset.user_ids))):
        for target_pos in target_item_pos:
            specific_cfs = _find_specific_cfs_(dataset, model, strategy, target_pos, False, sim_matrix, user_id, 100000, top_k)
            
            user_sequences = test.sequences[test.user_ids == user_id]

            if len(user_sequences) < 1 : 
                break
            if all(v > 0 for v in user_sequences[0]):
                original_interactions = user_sequences[0]
                best_interactions = specific_cfs[0].interactions['best']
                items_removed = np.setdiff1d(original_interactions, best_interactions)

                predictions = -pretrained_models['lstm'].predict(original_interactions)
                predictions[original_interactions] = StaticVars.FLOAT_MAX
                target_item = predictions.argsort()[min(top_k, target_pos)]
                predictions_reverse = -pretrained_models['lstm'].predict(best_interactions)
                predictions_reverse[best_interactions] = StaticVars.FLOAT_MAX
                pos_target_item_reverse = np.where(predictions_reverse.argsort() == target_item)[0][0] #bug maybe?
                worst_jaccard_sample = find_worst_items_with_jaccard(target_item, best_interactions, jaccard_sims_matrix, len_sample)
                jaccard_sample = find_best_items_with_jaccard(target_item, best_interactions, jaccard_sims_matrix, len_sample)
                rs_sample = find_best_items_using_recommender(target_item, best_interactions, pretrained_models['lstm'], len_sample)

                if len(items_removed) >= len_sample:
                    print(f"Sequence skipped, too much items removed for user_id {user_id} and n_sequence {n_sequence}.")
                    continue
                
                if not set(items_removed) <= set(jaccard_sample):
                    print("before jacc : ", jaccard_sample)
                    replace_items_if_missing(items_removed, jaccard_sample)
                    print("after jacc : ", jaccard_sample)
                    _jacc_search_info = retrieve_solutions_specific_sequence(user_id, test, pretrained_models['lstm'], get_backend_strategy('brute_force'), 100000, top_k, True, jaccard_sims_matrix, best_interactions, target_item, jaccard_sample)
                if not set(items_removed) <= set(rs_sample):
                    print("before rs : ", rs_sample)
                    replace_items_if_missing(items_removed, rs_sample)
                    print("after rs : ", rs_sample)
                    rs_search_info = retrieve_solutions_specific_sequence(user_id, test, pretrained_models['lstm'], get_backend_strategy('brute_force'), 100000, top_k, True, jaccard_sims_matrix, best_interactions, target_item, rs_sample)

                if not set(items_removed) <= set(worst_jaccard_sample):
                    print("before worst jacc : ", worst_jaccard_sample)
                    replace_items_if_missing(items_removed, worst_jaccard_sample)
                    print("after worst jacc : ", worst_jaccard_sample)
                    worst_jacc_search_info = retrieve_solutions_specific_sequence(user_id, test, pretrained_models['lstm'], get_backend_strategy('brute_force'), 100000, top_k, True, jaccard_sims_matrix, best_interactions, target_item, worst_jaccard_sample)

                evaluation_df = evaluation_df.append([user_id, target_pos, items_removed, len(items_removed)])
    return evaluation_df
create_reverse_mode_evaluation_dataframe(test, pretrained_models['lstm'], get_backend_strategy('brute_force'), [1, 3, 5, 7], 10, jaccard_sims_matrix)

  0%|          | 0/6039 [00:00<?, ?it/s]

[1277  461 1527  594  485  466  413  323 1257  448  440  508   90   28
  888  124  891  488  489  539]
before jacc :  [2731, 3523, 790, 3516, 3133, 3168, 1883, 2286, 3518, 3706, 3522, 3517, 3528, 2231, 1376, 3520, 186, 3527, 3519, 2634]
after jacc :  [257, 357, 266, 331, 123, 315, 1883, 2286, 3518, 3706, 3522, 3517, 3528, 2231, 1376, 3520, 186, 3527, 3519, 2634]
before rs :  [ 539  894 3629  117  366  466  124  485  594  146 2371   28 1586  888
 1636 2551 1550  461 3556  325]
after rs :  [ 257  357  266  331  123  315  124  485  594  146 2371   28 1586  888
 1636 2551 1550  461 3556  325]
before worst jacc :  [285, 505, 594, 19, 1578, 507, 492, 508, 480, 440, 293, 510, 485, 439, 325, 447, 266, 543, 521, 461]
after worst jacc :  [257, 357, 331, 315, 123, 507, 492, 508, 480, 440, 293, 510, 485, 439, 325, 447, 266, 543, 521, 461]
[1277  461 1527  594  485  466  413  323 1257  448  440  508   90   28
  888  124  891  488  489  539]
before jacc :  [3242, 1494, 3226, 3228, 3247, 2570, 2194, 

KeyboardInterrupt: 

In [127]:
import numpy as np
from collections import defaultdict
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file


FLOAT_MAX = np.finfo(np.float32).max

target_pos = 7
user_id = 8
top_k = 10
brute_force_specific_cfs = None
i = 0

# while brute_force_specific_cfs == [] or brute_force_specific_cfs is None:
#     i += 1
#     brute_force_specific_cfs = _find_specific_cfs_(test, pretrained_models['lstm'], get_backend_strategy('brute_force'), 7, False, jaccard_sims_matrix, i, 100000, 10)

brute_force_specific_cfs = _find_specific_cfs_(test, pretrained_models['lstm'], get_backend_strategy('brute_force'), target_pos, False, jaccard_sims_matrix, user_id, 100000, top_k)

user_sequences = test.sequences[test.user_ids == user_id]
user_sequences = [sequence for sequence in user_sequences if all(value > 0 for value in sequence)]

original_interactions = user_sequences[0]
print("original_interactions", original_interactions)

best_interactions = brute_force_specific_cfs[0].interactions['best']
print("best_interactions", best_interactions)

items_removed = np.setdiff1d(original_interactions, best_interactions)
print("items_removed", items_removed)

predictions = -pretrained_models['lstm'].predict(original_interactions)
predictions[original_interactions] = FLOAT_MAX
target_item = predictions.argsort()[min(top_k, target_pos)]
print("target_item", target_item)

predictions_reverse = -pretrained_models['lstm'].predict(best_interactions)
predictions_reverse[best_interactions] = FLOAT_MAX
pos_target_item_reverse = np.where(predictions_reverse.argsort() == target_item)[0][0]
print("position target item in reverse mode", pos_target_item_reverse)

worst_jaccard_sample = find_worst_items_with_jaccard(target_item, best_interactions, jaccard_sims_matrix, 20)
print("worst_jacc", worst_jaccard_sample)

jaccard_sample = find_best_items_with_jaccard(target_item, best_interactions, jaccard_sims_matrix, 20)
print("jacc", jaccard_sample)
rs_sample = find_best_items_using_recommender(target_item, best_interactions, pretrained_models['lstm'], 20)
if set(items_removed) <= set(jaccard_sample):
    print("jaccard_sample", jaccard_sample)
    search_info = retrieve_solutions_specific_sequence(user_id, test, pretrained_models['lstm'], get_backend_strategy('brute_force'), 100000, top_k, True, jaccard_sims_matrix, best_interactions, target_item, jaccard_sample)

elif set(items_removed) <= set(rs_sample):
    print("rs_sample", rs_sample)
    search_info = retrieve_solutions_specific_sequence(user_id, test, pretrained_models['lstm'], get_backend_strategy('brute_force'), 100000, top_k, True, jaccard_sims_matrix, best_interactions, target_item, rs_sample)
else:
    worst_jaccard_sample[:len(items_removed)] = items_removed[:len(items_removed)]
    print("modified_sample", worst_jaccard_sample)
    search_info = retrieve_solutions_specific_sequence(user_id, test, pretrained_models['lstm'], get_backend_strategy('brute_force'), 100000, top_k, True, jaccard_sims_matrix, best_interactions, target_item, worst_jaccard_sample)

print("search_info", search_info[0].interactions['best'])

result2 = dict.fromkeys([pos_target_item_reverse])
result2[pos_target_item_reverse] = []
result2[pos_target_item_reverse].extend(search_info)

cnt2 = defaultdict(dict)
no_target_achieved_cases2 = defaultdict(list)


cnt2, no_target_achieved_cases2 = convert_res_to_lists(result2, cnt2, no_target_achieved_cases2, "random_0", True)
print(cnt2)
print(no_target_achieved_cases2)

result = dict.fromkeys([target_pos])
result[target_pos] = []
result[target_pos].extend(brute_force_specific_cfs)

cnt = defaultdict(dict)
no_target_achieved_cases = defaultdict(list)

target_item = 0

cnt, no_target_achieved_cases = convert_res_to_lists(result, cnt, no_target_achieved_cases, "random_0", False)
print(cnt)
print(no_target_achieved_cases)

[2078 1439 2094 1360 1769 1329 1417   80 1443  704  591  890 1749 1381
  512 1408  504  451 1895  905]
original_interactions [384 461 236 478  86 457 511  60 323 443 360 473 472 514 456 481 119 450
 453 444]
best_interactions [384 461 236 478  86 457 511  60 323 443 360 473 472 514 456 481 119 450
 453]
items_removed [444]
target_item 80
position target item in reverse mode 12
worst_jacc [320, 84, 376, 1203, 1059, 223, 777, 91, 138, 196, 151, 121, 127, 75, 131, 118, 126, 896, 147, 80]
jacc [2939, 3595, 1233, 3231, 2655, 3328, 3468, 3706, 3602, 3603, 2446, 954, 3600, 2303, 3598, 24, 3601, 3597, 3471, 480]
modified_sample [444, 84, 376, 1203, 1059, 223, 777, 91, 138, 196, 151, 121, 127, 75, 131, 118, 126, 896, 147, 80]
search_info [384, 461, 236, 478, 86, 457, 511, 60, 323, 443, 360, 473, 472, 514, 456, 481, 119, 450, 453, 444]
[[1]]
[[1], [1]]
[[1], [1], [1]]
[[1], [1], [1], [1]]
[[1], [1], [1], [1], [2]]
[[1], [1], [1], [1], [2], [2]]
[[1], [1], [1], [1], [2], [2], [8]]
[[1], [1], [1],

In [128]:
methods_supported = ['Brute_Force', 'Random', 'BFS'] + \
    [f'BiDirectional_{m}' for m in ['001', '5', '999']] + \
    [f'Combo_{m}' for m in ['001', '5', '999']]

strategies = [('init', 2), ('best', 3)]
custom_range = [9, 29] # Range to access to the "stats per cardinality" part (please check the convert_res_to_lists method in the helpers notebook for more info)
target_pos = list(cnt.keys())
cnames = ['user_id', 'method', 'pos', 'budget', 'init_budget', 'best_budget', 'cardinality', 'cfs_orig', 'cfs']

rows, cols = 5, 1

tmp_dfs = []
for col in range(cols):
    for pos, row in zip(target_pos, range(rows)):

        tmp_dfs.append(pd.DataFrame.from_records(list(
            itertools.chain(*(
                zip(v[6], itertools.repeat(k), itertools.repeat(pos), v[i], v[2], v[3],
                    itertools.repeat(i - custom_range[0] + 1), v[7], v[8])
                for k, v in cnt[pos].items() if f'_{col}' in k and k.rsplit('_', 1)[0] in map(lambda x: x.lower(), methods_supported)
                for i in range(custom_range[0], custom_range[1])
            ))), columns=cnames
        ))

df = pd.concat(tmp_dfs, ignore_index=True)
print(df.head(20))

    user_id    method  pos  budget  init_budget  best_budget  cardinality  \
0         8  random_0    7      20           20           20            1   
1         8  random_0    7       0           20           20            2   
2         8  random_0    7       0           20           20            3   
3         8  random_0    7       0           20           20            4   
4         8  random_0    7       0           20           20            5   
5         8  random_0    7       0           20           20            6   
6         8  random_0    7       0           20           20            7   
7         8  random_0    7       0           20           20            8   
8         8  random_0    7       0           20           20            9   
9         8  random_0    7       0           20           20           10   
10        8  random_0    7       0           20           20           11   
11        8  random_0    7       0           20           20           12   

In [71]:
methods_supported = ['Brute_Force', 'Random', 'BFS'] + \
    [f'BiDirectional_{m}' for m in ['001', '5', '999']] + \
    [f'Combo_{m}' for m in ['001', '5', '999']]

strategies = [('init', 2), ('best', 3)]
custom_range = [9, 28] # Range to access to the "stats per cardinality" part (please check the convert_res_to_lists method in the helpers notebook for more info)
target_pos = list(cnt2.keys())
cnames = ['user_id', 'method', 'pos', 'budget', 'init_budget', 'best_budget', 'cardinality', 'cfs_orig', 'cfs']

rows, cols = 5, 1

tmp_dfs = []
for col in range(cols):
    for pos, row in zip(target_pos, range(rows)):

        tmp_dfs.append(pd.DataFrame.from_records(list(
            itertools.chain(*(
                zip(v[6], itertools.repeat(k), itertools.repeat(pos), v[i], v[2], v[3],
                    itertools.repeat(i - custom_range[0] + 1), v[7], v[8])
                for k, v in cnt2[pos].items() if f'_{col}' in k and k.rsplit('_', 1)[0] in map(lambda x: x.lower(), methods_supported)
                for i in range(custom_range[0], custom_range[1])
            ))), columns=cnames
        ))

df = pd.concat(tmp_dfs, ignore_index=True)
print(df.head(20))

    user_id    method  pos  budget  init_budget  best_budget  cardinality  \
0         8  random_0   12       0            3            3            1   
1         8  random_0   12       3            3            3            2   
2         8  random_0   12       0            3            3            3   
3         8  random_0   12       0            3            3            4   
4         8  random_0   12       0            3            3            5   
5         8  random_0   12       0            3            3            6   
6         8  random_0   12       0            3            3            7   
7         8  random_0   12       0            3            3            8   
8         8  random_0   12       0            3            3            9   
9         8  random_0   12       0            3            3           10   
10        8  random_0   12       0            3            3           11   
11        8  random_0   12       0            3            3           12   

In [61]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

backend = 'random'
random_cfs = [

    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 500, init_budget=1000)
    # _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000)
]

%store random_cfs


The backend used is: Random


target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

users loop:   0%|          | 0/10 [00:00<?, ?it/s]

target position loop: 20it [00:11,  1.72it/s]              

users loop:   0%|          | 0/10 [00:00<?, ?it/s]

...

users loop:   0%|          | 0/10 [00:00<?, ?it/s]

target position loop: 40it [00:32,  1.16it/s]

users loop:   0%|          | 0/10 [00:00<?, ?it/s]

target position loop: 40it [00:42,  1.05s/it]

Stored 'random_cfs' (list)


In [20]:
backend = 'bfs'

bfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 500, init_budget=1000),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 500, init_budget=1000, early_term=True),
]

# %store bfs_yloss_cfs_positive
%store bfs_yloss_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BFS


users loop:   0%|          | 0/10 [00:00<?, ?it/s]

target :  508
[-0.         -0.4886701  -1.2704934  ...  1.4422629   0.21140254
  0.10461938]
-0.0
0.104619384
3707
len ranked_items :  3686
top :  [3685 2757 3545 3544 3301 3542 3540 3539 3538 3536]
target :  890
[-0.        -1.7293239  0.7516235 ...  1.2451138  0.8454702  2.057247 ]
-0.0
2.057247
3707
len ranked_items :  3686
top :  [3685 3367 3368 3370 3371 3374 3376 3377 3378 3385]
target :  1015
[-0.         -0.01268113 -0.72315276 ...  1.8612478   1.2578342
  0.7910427 ]
-0.0
0.7910427
3707
len ranked_items :  3686
top :  [3685 3577 3576 3237 3571 3570 3565 3564 3245 3557]


target position loop: 20it [00:00, 35.74it/s]              

...

target position loop: 40it [00:01, 19.49it/s]

users loop:   0%|          | 0/10 [00:00<?, ?it/s]

target :  488
[-0.         -0.4886701  -1.2704934  ...  1.4422629   0.21140254
  0.10461938]
-0.0
0.104619384
3707
len ranked_items :  3686
top :  [3685 3450 3452 2465 3453 2120 3454 3455 3447 3457]
target :  451
[-0.        -1.7293239  0.7516235 ...  1.2451138  0.8454702  2.057247 ]
-0.0
2.057247
3707
len ranked_items :  3686
top :  [3685 3472 3218 3473 3216 3475 3476 3477 3471 3215]


target position loop: 40it [00:02, 16.96it/s]

target :  320
[-0.         -0.01268113 -0.72315276 ...  1.8612478   1.2578342
  0.7910427 ]
-0.0
0.7910427
3707
len ranked_items :  3686
top :  [3685 3582 3393 3581 3580 3579 3577 3576 3408 3415]
target :  65
[-0.         -2.4513376   0.45527014 ...  1.0769624   1.3857057
  1.618092  ]
-0.0
1.618092
3707
len ranked_items :  3686
top :  [3032 3603 3604 3605 3612 3616 3617 1785 3566 3633]
Stored 'bfs_yloss_cfs' (list)


In [21]:
backend='bidirectional'

bidirectional_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=500, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=500, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=500, init_budget=1000, alpha=0.999, normalization='default'),
]

%store bidirectional_cfs

target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: BiDirectional


users loop:   0%|          | 0/10 [00:00<?, ?it/s]

target :  508
[-0.         -0.4886701  -1.2704934  ...  1.4422629   0.21140254
  0.10461938]
-0.0
0.104619384
3707
len ranked_items :  3686
top :  [3685 2757 3545 3544 3301 3542 3540 3539 3538 3536]
target :  890
[-0.        -1.7293239  0.7516235 ...  1.2451138  0.8454702  2.057247 ]
-0.0
2.057247
3707
len ranked_items :  3686
top :  [3685 3367 3368 3370 3371 3374 3376 3377 3378 3385]


target position loop: 10it [00:08,  1.22it/s]              


KeyboardInterrupt: 

In [ ]:
backend='brute_force'

brute_force_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=500, init_budget=100000),
]

%store brute_force_cfs

In [ ]:
backend='combo'
combo_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=500, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=500, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=500, init_budget=1000, alpha=0.999, normalization='default'),
]

%store combo_cfs